In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Create a smaller dummy dataset
data = {
    'User': ['User1', 'User1', 'User2', 'User2', 'User3', 'User3'],
    'Item': ['Item1', 'Item2', 'Item1', 'Item2', 'Item1', 'Item3'],
    'Rating': [5, 3, 4, 1, 4, 2]
}

df = pd.DataFrame(data)

# Create a user-item matrix
user_item_matrix = df.pivot_table(index='User', columns='Item', values='Rating')
user_item_matrix = user_item_matrix.fillna(0)

print("User-Item Matrix:")
print(user_item_matrix)


User-Item Matrix:
Item   Item1  Item2  Item3
User                      
User1    5.0    3.0    0.0
User2    4.0    1.0    0.0
User3    4.0    0.0    2.0


In [2]:
# Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

# Convert the similarity matrix into a DataFrame
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

print("\nUser Similarity Matrix:")
print(user_similarity_df)



User Similarity Matrix:
User      User1     User2     User3
User                               
User1  1.000000  0.956674  0.766965
User2  0.956674  1.000000  0.867722
User3  0.766965  0.867722  1.000000


In [3]:
def predict_ratings(user, item):
    similar_users = user_similarity_df[user]
    user_ratings = user_item_matrix.loc[:, item]
    non_zero_similarities = similar_users[user_ratings > 0]

    if non_zero_similarities.sum() == 0:
        return 0
    
    predicted_rating = np.dot(non_zero_similarities, user_ratings[user_ratings > 0]) / non_zero_similarities.sum()
    return predicted_rating

# Predict ratings for all user-item pairs
predicted_ratings = user_item_matrix.copy()

for user in user_item_matrix.index:
    for item in user_item_matrix.columns:
        if user_item_matrix.loc[user, item] == 0:
            predicted_ratings.loc[user, item] = predict_ratings(user, item)

print("\nPredicted Ratings:")
print(predicted_ratings)

# Get top recommendations for each user
def recommend_items(user, num_recommendations=1):
    user_ratings = predicted_ratings.loc[user]
    already_rated = user_item_matrix.loc[user].drop(user_item_matrix.loc[user][user_item_matrix.loc[user] > 0].index)
    recommendations = user_ratings.drop(already_rated.index).nlargest(num_recommendations)
    return recommendations

# Example: Get top recommendation for 'User1'
user = 'User1'
recommendations = recommend_items(user, num_recommendations=1)

print(f"\nTop recommendation for {user}:")
print(recommendations)



Predicted Ratings:
Item   Item1     Item2  Item3
User                         
User1    5.0  3.000000    2.0
User2    4.0  1.000000    2.0
User3    4.0  1.938363    2.0

Top recommendation for User1:
Item
Item1    5.0
Name: User1, dtype: float64
